<a href="https://colab.research.google.com/github/leducthanhkim2004/aio2024/blob/main/Homework_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt
import torch
import torch .nn as nn
import torch .nn. functional as F
from torch . utils . data import Dataset , DataLoader
from sklearn . preprocessing import StandardScaler
from sklearn . model_selection import train_test_split

In [4]:
random_state = 59
np . random . seed ( random_state )
torch . manual_seed ( random_state )
if torch.cuda . is_available () :
    torch . cuda . manual_seed_all ( random_state )


In [5]:
dataset_path='/content/drive/MyDrive/Aio_data/Auto_MPG_data.csv'
dataset =pd.read_csv(dataset_path)
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
0,18.0,8,307.0,130.0,3504.0,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,0,0,1


In [6]:
X =dataset.drop(columns="MPG").values
y=dataset['MPG'].values

In [14]:
train_size=0.7
test_size=0.2
val_size=0.1
X_train, X_val,y_train,y_val=train_test_split(
    X,y,test_size=val_size,
    random_state=random_state,
    shuffle =True
)
X_train,X_test,y_train,y_test=train_test_split(
    X,y,test_size=test_size,
    random_state=random_state,
    shuffle=True
)


In [19]:
normalizer=StandardScaler()
X_train=normalizer.fit_transform(X_train)
X_val= normalizer.fit_transform(X_val)
X_test=normalizer.fit_transform(X_test)
# convert to tensor
X_train= torch.tensor(X_train,dtype=torch.float32)
X_val=torch.tensor(X_val,dtype=torch.float32)
X_test=torch.tensor(X_test,dtype=torch.float32)
y_train=torch.tensor(y_train,dtype=torch.float32)
y_val=torch.tensor(y_val,dtype=torch.float32)
y_test=torch.tensor(y_test,dtype=torch.float32)

<ipython-input-19-cfd0bb2bb75c>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train=torch.tensor(y_train,dtype=torch.float32)
<ipython-input-19-cfd0bb2bb75c>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val=torch.tensor(y_val,dtype=torch.float32)
<ipython-input-19-cfd0bb2bb75c>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test=torch.tensor(y_test,dtype=torch.float32)


In [29]:
class MPGDataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y
  def __len__(self):
    return len(self.y)
  def __getitem__(self,indx):
    return self.X[indx],self.y[indx]

In [30]:
batch_size=32
train_dataset=MPGDataset(X_train,y_train)
train_loader=DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
val_dataset=MPGDataset(X_val,y_val)
val_loader=DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [55]:
class MPGModel(nn.Module):
  def __init__(self,input_dims,hidden_dims,output_dims) :
    super().__init__()
    self.fc1=nn.Linear(input_dims,hidden_dims)
    self.fc2=nn.Linear(hidden_dims,hidden_dims)
    self.output=nn.Linear(hidden_dims,output_dims)
  def forward(self,X):
    x=F.relu(self.fc1(X))
    x=F.relu(self.fc2(x))
    x=self.output(x)
    return x.squeeze(1)


In [41]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [56]:
input_dims=X_train.shape[1]
hidden_dims=64
output_dims=1
model=MPGModel(input_dims,hidden_dims,output_dims).to(device)

In [57]:
model.forward(X_train)

tensor([-0.2090, -0.1823, -0.1200, -0.1938, -0.3697, -0.2342, -0.3028, -0.1169,
        -0.1174, -0.2028, -0.2247, -0.1944, -0.2055, -0.1915, -0.1142, -0.1570,
        -0.1513, -0.1631, -0.1577, -0.2123, -0.1279, -0.1645, -0.2242, -0.1188,
        -0.2566, -0.1719, -0.2006, -0.2819, -0.1788, -0.1404, -0.2123, -0.2196,
        -0.3024, -0.1825, -0.1706, -0.1882, -0.2120, -0.2341, -0.1451, -0.5535,
        -0.3724, -0.2101, -0.1659, -0.1670, -0.2473, -0.3499, -0.1173, -0.1163,
        -0.2163, -0.2059, -0.2135, -0.1366, -0.3782, -0.2103, -0.2419, -0.1926,
        -0.1816, -0.3069, -0.2202, -0.1804, -0.1586, -0.1659, -0.1597, -0.4099,
        -0.1210, -0.3262, -0.1748, -0.2464, -0.1910, -0.1919, -0.2041, -0.1549,
        -0.2122, -0.2122, -0.1484, -0.1513, -0.1395, -0.2069, -0.4668, -0.2233,
        -0.2750, -0.1650, -0.1955, -0.4219, -0.2160, -0.2110, -0.1803, -0.2333,
        -0.1589, -0.1910, -0.1389, -0.5496, -0.2249, -0.1352, -0.1592, -0.2400,
        -0.2163, -0.1168, -0.1724, -0.32

In [45]:
lr = 1e-2
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [65]:
def r_squared (y_true , y_pred ) :
  y_true = torch . Tensor ( y_true ) .to( device )
  y_pred = torch . Tensor ( y_pred ) .to( device )
  mean_true = torch . mean ( y_true )
  ss_tot = torch .sum (( y_true - mean_true ) ** 2)
  ss_res = torch .sum (( y_true - y_pred ) ** 2)
  r2 = 1 - ( ss_res / ss_tot )
  return r2

In [59]:
epochs = 100
train_losses = []
val_losses = []
train_r2 = []
val_r2 = []

In [66]:
for epoch in range(epochs):
    train_loss = 0.0
    train_target = []
    val_target = []
    train_predict = []
    val_predict = []
    model.train()  # Set the model to training mode
    for X_samples, y_samples in train_loader:
        X_samples = X_samples.to(device)
        y_samples = y_samples.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = model(X_samples)  # Forward pass
        train_predict += outputs.tolist()  # Collect predictions for R^2 calculation
        train_target += y_samples.tolist()  # Collect true targets for R^2 calculation
        loss = criterion(outputs, y_samples)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model weights
        train_loss += loss.item()  # Accumulate train loss
    train_loss /= len(train_loader)  # Average train loss over batches
    train_losses.append(train_loss)  # Store train loss for plotting
    train_r2.append( r_squared(train_target, train_predict))  # Calculate and store train R^2

    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():  # Disable gradient calculation during validation
        for X_samples, y_samples in val_loader:
            X_samples = X_samples.to(device)
            y_samples = y_samples.to(device)
            outputs = model(X_samples)  # Forward pass
            val_predict += outputs.tolist()  # Collect predictions for R^2 calculation
            val_target += y_samples.tolist()  # Collect true targets for R^2 calculation
            loss = criterion(outputs, y_samples)  # Calculate loss
            val_loss += loss.item()  # Accumulate validation loss
    val_loss /= len(val_loader)  # Average validation loss over batches
    val_losses.append(val_loss)  # Store validation loss for plotting
    val_r2.append(r_squared(val_target, val_predict))  # Calculate and store validation R^2

    print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train R^2: {train_r2[-1]:.4f}, Val R^2: {val_r2[-1]:.4f}")  # Print epoch metrics

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/100], Train Loss: 1.0545, Val Loss: 0.8841, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [2/100], Train Loss: 1.0635, Val Loss: 1.0309, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [3/100], Train Loss: 1.0554, Val Loss: 0.9378, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [4/100], Train Loss: 1.0444, Val Loss: 1.0440, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [5/100], Train Loss: 1.0535, Val Loss: 1.1390, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [6/100], Train Loss: 1.0556, Val Loss: 1.2013, Train R^2: -329.6122, Val R^2: -41.2928
Epoch [7/100], Train Loss: 1.0614, Val Loss: 1.0418, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [8/100], Train Loss: 1.0560, Val Loss: 0.8713, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [9/100], Train Loss: 1.0574, Val Loss: 1.1724, Train R^2: -329.6121, Val R^2: -41.2928
Epoch [10/100], Train Loss: 1.0427, Val Loss: 1.0426, Train R^2: -329.6122, Val R^2: -41.2928
Epoch [11/100], Train Loss: 1.0511, Val Loss: 1.3693, Train R^2: -329

In [69]:
model . eval ()
with torch . no_grad () :
  y_hat = model ( X_test )
  test_set_r2 = r_squared (y_hat , y_test )
  print ('Evaluation on test set :')
  print (f'R2: { test_set_r2 }')

Evaluation on test set :
R2: -87174.5
